In [1]:
import math
import numpy as np
import pandas as pd
from oucp_utils import *
from scipy.optimize import fsolve
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

In [2]:
# carbon content per unit of energy
h1 = 1

# elasticity parameters
sigma = 1
sigmaE = 1
theta = 4
rho = 0
alpha = 0.15

scenario = 'opt'

# elasticity of supply of renewables
epsilonSstar1 = 0.5
epsilonS1 = 0.5

# baseline, no renewable energy
epsilonSvec = [(epsilonS1, h1, 1)]
epsilonSstarvec = [(epsilonSstar1, h1, 1)]
assert(sum(k for i,j,k in epsilonSvec) == 1)
assert(sum(k for i,j,k in epsilonSstarvec) == 1)


data = pd.read_excel("../../raw_data/baselinecarbon_direct_2018.xlsx")
#data = pd.read_csv("../../raw_data/BaselineCarbon_2015.csv")
#data['Cey'] = data['Cey'] + data['Ced']
#data['Ceystar'] = data['Ceystar'] + data['Cedstar']
#data['Ced'] = 0
#data['Cedstar'] = 0
data['jxbar']=data['Cex']/(data['Cex'] + data['Ceystar'])
data['jmbar']=data['Cey']/(data['Cey'] + data['Cem'])

if scenario == 'constrained':
    data = data[(data['region_scenario'] == 3)] 
    tax_scenario= ['purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid']
elif scenario == 'global':
    data = data[(data['region_scenario'] == 3)] 
    tax_scenario= ['global']
elif scenario == 'opt':
    data = data[data['region_scenario'] != 4]
    tax_scenario= ['Unilateral']
#tax_scenario = ['Unilateral']
data = data[(data['region_scenario'] == 3)]
tax_scenario = ['EPC_hybrid']

In [3]:
def max_welfare(p, phi, tax, region_data):
    return -model.comp_welfare(p, phi, tax,region_data)

In [4]:
phi_list = np.arange(0,2,0.1)
model_parameters = (theta, sigma, sigmaE, epsilonSvec, epsilonSstarvec, rho, alpha)
model = taxModel(data, tax_scenario, phi_list, model_parameters)

In [5]:
index, region_data = next(model.data.iterrows())
tax = 'Unilateral'
phi = 2

In [6]:
res = minimize(max_welfare, [1,0.8, 0, 1], args = (phi, tax, region_data), method='nelder-mead', tol=0.000001, options={'maxfev': 1000}, bounds = [(0.01,np.inf), (0, np.inf), (0,1), (2000000,np.inf)])
res

 final_simplex: (array([[1.07466431e+00, 8.43352704e-01, 4.27875134e-05, 2.00000000e+06],
       [1.07466431e+00, 8.43352704e-01, 4.27875136e-05, 2.00000000e+06],
       [1.07466431e+00, 8.43352704e-01, 4.27875136e-05, 2.00000000e+06],
       [1.07466431e+00, 8.43352704e-01, 4.27875134e-05, 2.00000000e+06],
       [1.07466431e+00, 8.43352704e-01, 4.27875133e-05, 2.00000000e+06]]), array([-10.64742075, -10.64742062, -10.64742047, -10.64742031,
       -10.64741976]))
           fun: -10.647420751228275
       message: 'Optimization terminated successfully.'
          nfev: 301
           nit: 166
        status: 0
       success: True
             x: array([1.07466431e+00, 8.43352704e-01, 4.27875134e-05, 2.00000000e+06])

In [7]:
arr = res.x
arr[-1] = 0
model.comp_welfare(arr, phi, tax,region_data) 

10.647421426243874

In [8]:
model.comp_obj(arr[0], 2, arr[1:3], phi, tax, region_data)

(-3.375077994860476e-13, 0.281323425178893, 0)

In [9]:
model.comp_obj(1.05739466576942, 2, [0.8465, 0], phi, tax, region_data)

(-0.5452638053042485, 0.705822462287792, 0)

In [10]:
model.comp_welfare([1.05739466576942,0.8465,0, 0], phi, tax,region_data) 

12.031322685403126

In [7]:
def obj_fn_te(p, phi, tax, region_data):
    diff, diff1 = model.te_obj(p, phi, tax, region_data)
    return abs(diff) + abs(diff1)

def obj_fn(p, phi, tax, region_data):
    diff, diff1, diff2 = model.obj_system(p, phi, tax, region_data)
    return abs(diff) + abs(diff1) + abs(diff2)

 final_simplex: (array([[0.85664173, 0.81586509, 0.52752612],
       [0.85664173, 0.81586507, 0.5275261 ],
       [0.85664171, 0.81586504, 0.52752613],
       [0.85664172, 0.8158651 , 0.52752609]]), array([1.14945806e-06, 1.60314493e-06, 1.74129998e-06, 1.85279859e-06]))
           fun: 1.149458057625452e-06
       message: 'Optimization terminated successfully.'
          nfev: 179
           nit: 98
        status: 0
       success: True
             x: array([0.85664173, 0.81586509, 0.52752612])

In [10]:
init_guess = np.random.rand(1000, 2) * 10000
result = []
for pe, tb in init_guess:
    #res = model.solve_obj(phi, tax, region_data, init_guess = [pe,tb, 0], verbose = False, second_try = False)
    res = minimize(obj_fn, [pe,tb], args = (phi, tax, region_data), method='nelder-mead', tol=0.000001, options={'maxfev': 100000})
    result.append((res.x,res.fun, (pe,tb)))

In [13]:
for res in result:
    if not (np.isclose(abs(res[0]), [1.24094625,1.33333333])).all():
        print(res)
    #if res[1] == 1:
    #    print(res)

(array([ 1.09860833, -0.79654207]), 26.688391712445423, (5.699311173572585, 5093.960546141944))


In [15]:
#model.solve_obj(2, tax, region_data, init_guess = [1,0,0], verbose = False)
fsolve(model.te_obj, [1,2], args = (phi, tax, region_data), full_output = True, maxfev = 100000)

(array([1.24094625, 1.33333333]),
 {'nfev': 10,
  'fjac': array([[-0.92146996,  0.38844962],
         [-0.38844962, -0.92146996]]),
  'r': array([-44.30783921, -21.44420869,  30.04099335]),
  'qtf': array([7.19505380e-08, 9.34561708e-08]),
  'fvec': array([-7.13491488e-11, -4.09308143e-11])},
 1,
 'The solution converged.')

In [10]:
init_guess = np.random.rand(1000, 2) * 10000
result = []
for pe, tb in init_guess:
    #res = model.solve_obj(phi, tax, region_data, init_guess = [pe,tb, 0], verbose = False, second_try = False)
    res = fsolve(model.te_obj, [pe,tb], args = (phi, tax, region_data), full_output = True, maxfev = 100000)
    result.append((res[0],res[2], (pe,tb)))

In [12]:
for res in result:
    #if not (np.isclose(abs(res[0]), [1.24094625,1.33333333])).all():
    #    print(res)
    if res[1] == 1:
        print(res)

In [11]:
result

[(array([8193.71260529, 7816.46538693, 2262.19882206]),
  5,
  (8193.712605291976, 7816.4653869274625)),
 (array([6759.29160005,  216.96671721, 9650.45690053]),
  5,
  (6759.291600045481, 216.96671720899508)),
 (array([4.46061521e+00, 1.55786389e+03, 4.97979712e+03]),
  5,
  (4.4606152098281004, 1557.8638919314703)),
 (array([4842.35893198, 1734.55236769, 8063.03987127]),
  5,
  (4842.358931975239, 1734.5523676931607)),
 (array([1021.89209244, 8636.86061184, 3340.38460518]),
  5,
  (1021.8920924429598, 8636.860611844151)),
 (array([8349.30345732, 2842.23684872, 9023.85765365]),
  5,
  (8349.303457324855, 2842.2368487225826)),
 (array([1308.28600649, 6920.65620348, 6956.29018189]),
  5,
  (1308.2860064872325, 6920.6562034831095)),
 (array([8173.40498645, 4074.63564739, 8412.86834838]),
  5,
  (8173.404986454778, 4074.635647385143)),
 (array([7913.3682206 , 9185.84578588, 1299.18247912]),
  5,
  (7913.368220599172, 9185.845785883044)),
 (array([5861.46856471, 8420.03164841, 9416.63650381

In [9]:
vec = np.array([2.6,1.1,0])
model.obj_system(vec, phi, tax, region_data)

array([1.07527398, 0.83860897, 0.        ])

In [7]:
df = model.retrieve()
df

,region,tax_sce,varphi,pe,tb,prop,te,jxbar_prime,jmbar_prime,j0_prime,...,delta_Vgstar,delta_UCed,delta_UCedstar,leak,leakstar,welfare,welfare_noexternality,Qe1_prime,Qe1star_prime,conv
0,OECD37 as Home,purete,0.0,1.000000,0.000000,0.000000,0.000000,0.057913,0.779597,0.057913,...,-0.000000,-5.611511e-16,-4.803935e-16,0.0,0.0,-5.551191e-14,-5.551191e-14,9.303970,24.331330,1
1,OECD37 as Home,purete,0.1,1.006892,0.000000,0.000000,0.073237,0.057913,0.779597,0.057913,...,-0.121693,-1.735774e-02,-1.485971e-02,0.0,0.0,1.532460e-02,-1.543617e-02,8.990040,24.415032,1
2,OECD37 as Home,purete,0.2,1.014039,0.000000,0.000000,0.146056,0.057913,0.779597,0.057913,...,-0.247013,-3.523280e-02,-3.016230e-02,0.0,0.0,6.175372e-02,-6.268289e-02,8.668100,24.501529,1
3,OECD37 as Home,purete,0.3,1.021464,0.000000,0.000000,0.218436,0.057913,0.779597,0.057913,...,-0.376272,-5.366969e-02,-4.594586e-02,0.0,0.0,1.400133e-01,-1.432837e-01,8.337458,24.591067,1
4,OECD37 as Home,purete,0.4,1.029193,0.000000,0.000000,0.290350,0.057913,0.779597,0.057913,...,-0.509830,-7.271978e-02,-6.225437e-02,0.0,0.0,2.508934e-01,-2.589937e-01,7.997316,24.683925,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,OECD37 as Home,EPC_hybrid,1.5,1.022819,0.591673,0.473661,1.500000,0.049801,0.779597,0.044658,...,-0.652939,-1.210580e+00,-4.881354e-02,0.138272135564499,0.473661483065879,5.653725e+00,-6.130237e+00,3.148153,24.607369,1
136,OECD37 as Home,EPC_hybrid,1.6,1.034398,0.637503,0.479661,1.600000,0.049452,0.779597,0.044056,...,-0.863261,-1.298884e+00,-7.316939e-02,0.139163987729774,0.479661184365634,6.473164e+00,-7.196150e+00,2.494809,24.746270,1
137,OECD37 as Home,EPC_hybrid,1.7,1.049192,0.685964,0.485475,1.700000,0.049119,0.779597,0.043480,...,-1.125183,-1.392732e+00,-1.038912e-01,0.140042706340715,0.485475292809093,7.365013e+00,-8.462427e+00,1.744473,24.922595,1
138,OECD37 as Home,EPC_hybrid,1.8,1.069323,0.738618,0.491076,1.800000,0.048805,0.779597,0.042932,...,-1.471700,-1.496579e+00,-1.450110e-01,0.140903166514635,0.491076246711533,8.339718e+00,-1.004788e+01,0.829103,25.160565,1
